In [ ]:
from __future__ import print_function, division
import tensorflow as tf
import tensorflow.keras as K 
import tensorflow.keras.layers as L
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Reshape
# from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from sklearn.utils import shuffle
# from deformableConv_op import DeformableConv
import os

In [2]:
# CNN-stream

In [3]:
# Setting the hyper-parameters
NUM_CLASS = 3
CHANNEL_AXIS = -1

BATCH_SIZE = 10  # 16 8
train_epochs = 200

In [8]:

def Enet_stream():


    Enet_block = K.applications.EfficientNetB2(include_top=None,
                                                weights='imagenet',
                                                input_tensor=None,
                                                input_shape=(224,224,3),
                                                pooling=None,
                                                # classes=3,
                                                classifier_activation="sigmoid",
                                                )
    x = Enet_block.output

    # x = layers.Flatten()(x)
    gap = L.GlobalAveragePooling2D()(x)
    x = L.Dense(128,activation='relu')(gap)
    output_tensor = L.Dense(NUM_CLASS, activation = 'softmax')(x)

    Enet_model = K.models.Model(inputs=Enet_block.input, outputs=output_tensor)

    return Enet_model

In [9]:
# ViT-stream

In [10]:
from vit_keras import vit, utils

In [12]:
image_size = 224

In [13]:
def get_ViT_model():
    
    base_model = vit.vit_b32(
        image_size=image_size,
        activation='sigmoid',
        pretrained=True, ##
        include_top=False,
        pretrained_top=False, 
        classes=3)
    
    x = base_model.output
    
    x = L.Dense(128, activation='relu')(x)  # FC
    predictions = L.Dense(NUM_CLASS, activation='softmax', kernel_initializer='he_normal',
                          kernel_regularizer=regularizers.l2(0.0001))(x)  # softmax
    
    model_ViT = K.models.Model(inputs=base_model.input, outputs=predictions)
    
    return model_ViT

In [14]:
# SE
def activation(x, func='relu'):
    
    return L.Activation(func)(x)

def squeeze_excitation_layer(x, out_dim=512):
    
    #squeeze = GlobalAveragePooling2D()(x)
    
    excitation = L.Dense(units=out_dim//4)(x)
    excitation = activation(excitation)
    excitation = L.Dense(units=out_dim)(excitation)
    excitation = activation(excitation, 'sigmoid')
    #excitation = Reshape((1,1,out_dim))(excitation)
    
    scale = L.multiply([x,excitation])
    
    return scale

In [15]:
# define the dual-stream

In [19]:
def DSDNN():    

    model_CNN = Enet_stream()
    model_ViT = get_ViT_model()
    
    CNN_out = model_CNN.layers[-3].output
    CNN_in = model_CNN.input
    
    ViT_out = model_ViT.layers[-3].output
    ViT_in = model_ViT.input
    
    # SE
    # CNN_out_SE = squeeze_excitation_layer(CNN_out, out_dim=1280) #B0 B1
    CNN_out_SE = squeeze_excitation_layer(CNN_out, out_dim=1408) #B2
#     CNN_out_SE = squeeze_excitation_layer(CNN_out, out_dim=1536) #B3
    # SE
    ViT_out_SE = squeeze_excitation_layer(ViT_out, out_dim=768)
    
    # merge
    merge = L.concatenate([CNN_out_SE, ViT_out_SE], axis=-1)
    merge = L.BatchNormalization(axis=-1)(merge)
    
    merge = Dense(units=128, activation='relu', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(merge)
    merge = L.Dropout(0.5)(merge)
    
    logits = L.Dense(NUM_CLASS, activation='softmax', kernel_initializer='he_normal',
                    kernel_regularizer=regularizers.l2(0.0001))(merge)
    
    model = K.models.Model([CNN_in, ViT_in], logits)
    
    optm = K.optimizers.Adam(lr=1e-4)
    
    model.compile(optimizer=optm, loss=['categorical_crossentropy'], metrics=['acc'])
    
    return model

In [20]:
def train_full(model, train_epochs, Y_train_path, Y_val_path, X_train_path, X_val_path):

    Y_train = tf.keras.utils.to_categorical(np.load(Y_train_path))
    Y_val = tf.keras.utils.to_categorical(np.load(Y_val_path))
    
    X_train = np.load(X_train_path)
    X_train_cp = X_train.copy()
    
    X_val = np.load(X_val_path)
    X_val_cp = X_val.copy()

    #scale
    # X_train = X_train/255.0
    # X_train_cp = X_train_cp/255.0

    # X_val = X_val/255.0
    # X_val_cp = X_val_cp/255.0

    X_train = K.applications.efficientnet.preprocess_input(X_train)
    X_train_cp = vit.preprocess_inputs(X_train_cp)
    
    ## vit-keras stream
    # X_val = X_val/255.0
    X_val = K.applications.efficientnet.preprocess_input(X_val)
    X_val_cp = vit.preprocess_inputs(X_val_cp)
    
    # shuffle
    X_train, X_train_cp, Y_train = shuffle(X_train, X_train_cp, Y_train)
    
    model_checkpoint = ModelCheckpoint('weight/full_weights_EnetB2_fine-tune_0731_train_test_2110PM_preprocess.h5', monitor='val_loss', verbose=1, 
                                       save_best_only=True, save_weights_only=True)
    
    early_stopping =EarlyStopping(monitor='val_loss', patience=20)
    
    model.fit([X_train, X_train_cp], Y_train, batch_size=BATCH_SIZE, epochs=train_epochs, verbose=1, 
              validation_data=([X_val, X_val_cp], Y_val), shuffle=True,
              callbacks=[model_checkpoint, early_stopping])

In [21]:
X_train_path = r'/home/rs/MachineLearning/LangFang/LangF_Fw_2021-06-25_train_dual_stream/train_X.npy'
Y_train_path = r'/home/rs/MachineLearning/LangFang/LangF_Fw_2021-06-25_train_dual_stream/train_Y.npy'
X_val_path = r'/home/rs/MachineLearning/LangFang/LangF_Fw_2021-06-25_train_dual_stream/valid_X.npy'
Y_val_path = r'/home/rs/MachineLearning/LangFang/LangF_Fw_2021-06-25_train_dual_stream/valid_Y.npy'

In [ ]:
model_dscnn = DSDNN()

In [24]:
train_epochs = 200

In [25]:
# start training
# train_full(model_dscnn, train_epochs, Y_train_path, Y_val_path, X_train_path, X_val_path)

In [ ]:
# start training
train_full(model_dscnn, train_epochs, Y_train_path, Y_val_path, X_train_path, X_val_path)

In [26]:
# Calculate OA and Kappa

In [26]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, cohen_kappa_score

In [27]:
def get_OA(cm):
    
    total = cm.sum()
    
    diag_arr = np.diagonal(cm, offset=0)
    
    correct = diag_arr.sum()
    
    OA = (correct/total) * 100
    
    return OA

def get_cm_oa_kappa(Y_test, Y_pred):
    
    Y_pred = np.argmax(Y_pred, axis=1)
    
    Y_test = Y_test.tolist()
    Y_pred = Y_pred.tolist()
    
    cm = confusion_matrix(Y_pred, Y_test)
    
    oa = get_OA(cm)
    
    kappa = cohen_kappa_score(Y_pred, Y_test)
    
    print(cm)
    
    return cm, oa, kappa

def load_npz(npz_path):
    
    npz_arr = np.load(npz_path)
    _files = npz_arr.files
    
    return npz_arr[_files[0]]

In [28]:
X_test_path = '/home/rs/MachineLearning/LangFang/LangF_Fw_2021-06-25_train_dual_stream/test_X.npz'
Y_test_path = '/home/rs/MachineLearning/LangFang/LangF_Fw_2021-06-25_train_dual_stream/test_Y.npy'

X_test = load_npz(X_test_path)
Y_test = np.load(Y_test_path)

X_test_cp = X_test.copy()

X_test = K.applications.efficientnet.preprocess_input(X_test)
X_test_cp = vit.preprocess_inputs(X_test_cp)
# X_test_cp = X_test_cp/255.0



In [29]:
_weights = 'weight/full_weights_EnetB2_fine-tune_0731_train_test_2110PM_preprocess.h5'

model = DSDNN()
model.load_weights(_weights)

In [ ]:
Y_pred = model.predict([X_test, X_test_cp], verbose=1)

In [ ]:
cm, oa, kappa = get_cm_oa_kappa(Y_test, Y_pred)
print('acc: {:.2f}%  Kappa: {:.4f}'.format(oa,kappa))